<a href="https://colab.research.google.com/github/wikibook/machine-learning/blob/2.0/jupyter_notebook/6.5.2_사전학습된_임베딩없이_영화리뷰_분류하기.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
import numpy as np
import pandas as pd

In [3]:
NUM_WORDS=10000 # only use top 1000 words
INDEX_FROM=3   # word index offset

(train_x,train_y),(test_x,test_y) = imdb.load_data(num_words=NUM_WORDS, index_from=INDEX_FROM)
train_x = train_x[:5000]
train_y = train_y[:5000]
test_x = test_x[:5000]
test_y = test_y[:5000]

In [4]:
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
word_to_id["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_id.items()}

In [5]:
print(' '.join(id_to_word[id] for id in train_x[0] ))

<START> this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert <UNK> is an amazing actor and now the same being director <UNK> father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for <UNK> and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also <UNK> to the two little boy's that played the <UNK> of norman and paul they were just brilliant children are often left out of the <UNK> list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for wha

In [6]:
print("train_x count: ", train_x.shape[0])
print("test_x count: ", test_x.shape[0])

train_x count:  5000
test_x count:  5000


In [7]:
print("Max value of a word index ")
print(max([max(sequence) for sequence in train_x]))
print("Max length num words of review in train ")
print(max([len(sequence) for sequence in train_x]))

Max value of a word index 
9999
Max length num words of review in train 
1851


In [8]:
max_words = 500
train_x = sequence.pad_sequences(train_x, maxlen=max_words)
test_x = sequence.pad_sequences(test_x, maxlen=max_words)

In [9]:
train_x.shape

(5000, 500)

In [10]:
model = Sequential()
model.add(Embedding(NUM_WORDS, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [14]:
print('Train...')
batch_size = 32
model.fit(train_x, train_y,
          batch_size=batch_size,
          epochs=5,
          validation_split = 0.1)
score, acc = model.evaluate(test_x, test_y,
                            batch_size=batch_size)


Train...
Train on 4500 samples, validate on 500 samples
Epoch 1/5
4500/4500 [==============================] - 106s 24ms/sample - loss: 0.4678 - accuracy: 0.7922 - val_loss: 0.5022 - val_accuracy: 0.7480
Epoch 2/5
4500/4500 [==============================] - 111s 25ms/sample - loss: 0.3043 - accuracy: 0.8747 - val_loss: 0.5423 - val_accuracy: 0.7720
Epoch 3/5
4500/4500 [==============================] - 106s 24ms/sample - loss: 0.1765 - accuracy: 0.9353 - val_loss: 0.5884 - val_accuracy: 0.8020
Epoch 4/5
4500/4500 [==============================] - 104s 23ms/sample - loss: 0.1275 - accuracy: 0.9558 - val_loss: 0.8244 - val_accuracy: 0.7600
Epoch 5/5
4500/4500 [==============================] - 107s 24ms/sample - loss: 0.0878 - accuracy: 0.9704 - val_loss: 0.8108 - val_accuracy: 0.7740
5000/1 [====================================================================================================================================================================================================

Test score: 0.7203288818001747
Test accuracy: 0.7906


In [15]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.7203288818001747
Test accuracy: 0.7906
